In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -r /content/Face-recognition-in-Near-InfraRed-images/requirements.txt
!pip install --upgrade opencv-python

In [ ]:
# import all libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K
import os
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
path=os.getcwd()
face_detection_path=os.getcwd(),"../"
print(os.getcwd())

/home/chinmay/sem-six/ai-ml/face_detection/face_detection_and_recongition_using_ir_camera/face_recognition


In [ ]:
### importing all the face detetion methods from face_detection folder
import sys
import os
sys.path.append("/content/Face-recognition-in-Near-InfraRed-images/")
from face_detection.yoloface_face_detection import yoloface_detection

importing face detection libraries....
now you can use face detection libraries


In [24]:
# define vgg face model's architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))	
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [ ]:
!unzip /content/vgg_face_weights.zip

In [25]:
# load vgg face model weights
# Remove last Softmax layer and get model upto last flatten layer #with outputs 2622 units 
model.summary()
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)
model.load_weights('vgg_face_weights.h5')


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_13 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 112, 112, 128)    

In [ ]:
# unzip dataset
!unzip /content/TD_NIR_A_Set.zip

In [ ]:
!python /content/Face-recognition-in-Near-InfraRed-images/face_detection/main.py

Data augmentation

In [ ]:
train_datasets = os.path.join(path, "/content/yolo_cropped_images")

In [ ]:
# Định nghĩa augmentation
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.MotionBlur(blur_limit=3, p=0.2),
    A.Affine(scale=(0.95, 1.05), rotate=(-5, 5), translate_percent=(0.02, 0.05), p=0.4),
    A.ToGray(p=1.0)  # Giữ ảnh ở dạng grayscale (NIR)
])

for folder in os.listdir(train_datasets):
    # Thư mục chứa ảnh gốc
    input_folder = os.path.join(train_datasets, folder)

    # Lặp qua tất cả ảnh trong thư mục đầu vào
    for filename in os.listdir(input_folder):
        
        if filename.startswith("AUG_"):
            continue

        input_path = os.path.join(input_folder, filename)
        
        # Đọc ảnh ở dạng grayscale
        image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Không thể đọc {filename}, bỏ qua...")
            continue

        # Augment ảnh
        augmented = transform(image=image)['image']
        
        # Lưu ảnh đã xử lý
        output_path = os.path.join(input_folder, "AUG_" + filename)
        cv2.imwrite(output_path, augmented)
        print(f"Lưu ảnh augment: {output_path}")

print("Hoàn thành augmentation!")

In [ ]:
import shutil
### python script to automatically divide data from tested_dataset folder
data_set_path=os.path.join(path,"/content/yolo_cropped_images")

images_crop_path=os.path.join(path,"/content/images_train_crop")
images_test_path=os.path.join(path,"/content/images_test_crop")

for file in os.listdir(data_set_path):

    ## create a folder inside images_crop and images_test_crop
    cur_crop_path=os.path.join(images_crop_path,file)
    cur_test_path=os.path.join(images_test_path,file)
    os.makedirs(cur_crop_path, exist_ok=True)
    os.makedirs(cur_test_path, exist_ok=True)

    specific_dataset_path=os.path.join(data_set_path,file)
    dataset=os.listdir(specific_dataset_path)

    partition=(len(dataset)*7)//10

    for i in range(partition):
        shutil.copy(os.path.join(specific_dataset_path,dataset[i]), os.path.join(cur_crop_path,dataset[i]))
    for i in range(partition,len(dataset)):
        shutil.copy(os.path.join(specific_dataset_path,dataset[i]), os.path.join(cur_test_path,dataset[i]))

FileExistsError: [Errno 17] File exists: '/home/chinmay/sem-six/ai-ml/face_detection/face_detection_and_recongition_using_ir_camera/face_recognition/images_crop/subject_3'

In [ ]:
# Prepare Training Data
x_train=[]
y_train=[]
person_rep=dict()
person_folders=os.listdir('/content/images_train_crop')
for i,person in enumerate(person_folders):
  person_rep[i]=person
  image_names=os.listdir('/content/images_train_crop/'+person+'/')
  print(person_rep[i])
  for image_name in image_names:
    print(image_name,end="\t")
    img=load_img('/content/images_train_crop/'+person+'/'+image_name,target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=preprocess_input(img)
    img_encode=vgg_face(img)
    x_train.append(np.squeeze(K.eval(img_encode)).tolist())
    y_train.append(i)


print(person_rep)


subject_3
96.jpg	100.jpg	104.jpg	109.jpg	134.jpg	85.jpg	18.jpg	55.jpg	1.jpg	73.jpg	14.jpg	84.jpg	181.jpg	56.jpg	62.jpg	94.jpg	147.jpg	77.jpg	72.jpg	37.jpg	154.jpg	58.jpg	112.jpg	12.jpg	170.jpg	102.jpg	142.jpg	116.jpg	80.jpg	16.jpg	182.jpg	108.jpg	60.jpg	81.jpg	141.jpg	155.jpg	23.jpg	69.jpg	180.jpg	163.jpg	22.jpg	38.jpg	145.jpg	143.jpg	135.jpg	187.jpg	74.jpg	30.jpg	11.jpg	186.jpg	95.jpg	0.jpg	33.jpg	91.jpg	136.jpg	127.jpg	128.jpg	4.jpg	88.jpg	21.jpg	92.jpg	46.jpg	43.jpg	133.jpg	6.jpg	70.jpg	98.jpg	54.jpg	13.jpg	169.jpg	105.jpg	34.jpg	68.jpg	76.jpg	89.jpg	113.jpg	83.jpg	152.jpg	164.jpg	99.jpg	115.jpg	65.jpg	125.jpg	3.jpg	61.jpg	51.jpg	59.jpg	124.jpg	185.jpg	90.jpg	183.jpg	24.jpg	129.jpg	157.jpg	25.jpg	93.jpg	118.jpg	165.jpg	17.jpg	168.jpg	2.jpg	144.jpg	64.jpg	28.jpg	40.jpg	5.jpg	15.jpg	50.jpg	71.jpg	107.jpg	20.jpg	8.jpg	36.jpg	110.jpg	149.jpg	78.jpg	139.jpg	138.jpg	126.jpg	166.jpg	137.jpg	57.jpg	153.jpg	82.jpg	9.jpg	106.jpg	123.jpg	48.jpg	39.jpg	160.jpg	63.jpg	subject_9
96.jpg	100.jpg	10

195.jpg	282.jpg	96.jpg	100.jpg	104.jpg	109.jpg	134.jpg	85.jpg	18.jpg	241.jpg	55.jpg	1.jpg	199.jpg	289.jpg	250.jpg	73.jpg	14.jpg	84.jpg	236.jpg	181.jpg	56.jpg	62.jpg	94.jpg	147.jpg	77.jpg	72.jpg	37.jpg	154.jpg	215.jpg	58.jpg	112.jpg	12.jpg	170.jpg	102.jpg	209.jpg	142.jpg	262.jpg	116.jpg	219.jpg	197.jpg	80.jpg	217.jpg	16.jpg	268.jpg	182.jpg	256.jpg	108.jpg	280.jpg	60.jpg	81.jpg	141.jpg	155.jpg	23.jpg	69.jpg	190.jpg	204.jpg	180.jpg	294.jpg	163.jpg	210.jpg	274.jpg	22.jpg	38.jpg	145.jpg	143.jpg	135.jpg	187.jpg	224.jpg	229.jpg	228.jpg	203.jpg	216.jpg	74.jpg	30.jpg	11.jpg	186.jpg	233.jpg	95.jpg	252.jpg	0.jpg	33.jpg	266.jpg	269.jpg	293.jpg	91.jpg	136.jpg	230.jpg	276.jpg	127.jpg	285.jpg	128.jpg	221.jpg	4.jpg	245.jpg	244.jpg	88.jpg	231.jpg	21.jpg	92.jpg	238.jpg	218.jpg	227.jpg	46.jpg	255.jpg	248.jpg	43.jpg	292.jpg	133.jpg	291.jpg	191.jpg	222.jpg	6.jpg	70.jpg	98.jpg	54.jpg	13.jpg	169.jpg	273.jpg	257.jpg	105.jpg	34.jpg	68.jpg	76.jpg	89.jpg	113.jpg	226.jpg	299.jpg	83.jpg	152.jpg	189.jpg	243.jpg	284

In [ ]:
person_rep

{0: 'subject_3',
 1: 'subject_9',
 2: 'subject_17',
 3: 'subject_14',
 4: 'subject_2',
 5: 'subject_10',
 6: 'subject_16',
 7: 'subject_11',
 8: 'subject_4',
 9: 'subject_15',
 10: 'subject_13'}

In [ ]:
#Prepare Test Data
x_test=[]
y_test=[]
person_folders=os.listdir('/content/images_test_crop')
for i,person in enumerate(person_folders):
  image_names=os.listdir('/content/images_test_crop/'+person)
  print(person)
  for image_name in image_names:
    print(image_name,end="\t")
    img=load_img('/content/images_test_crop/'+person+'/'+image_name,target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=preprocess_input(img)
    img_encode=vgg_face(img)
    x_test.append(np.squeeze(K.eval(img_encode)).tolist())
    y_test.append(i)
  print()

subject_3
131.jpg	79.jpg	29.jpg	31.jpg	27.jpg	114.jpg	75.jpg	148.jpg	103.jpg	32.jpg	41.jpg	167.jpg	175.jpg	119.jpg	176.jpg	162.jpg	159.jpg	158.jpg	47.jpg	121.jpg	42.jpg	45.jpg	172.jpg	87.jpg	184.jpg	53.jpg	86.jpg	178.jpg	10.jpg	19.jpg	171.jpg	130.jpg	111.jpg	35.jpg	7.jpg	26.jpg	151.jpg	67.jpg	156.jpg	140.jpg	174.jpg	120.jpg	173.jpg	150.jpg	177.jpg	161.jpg	132.jpg	52.jpg	49.jpg	179.jpg	117.jpg	101.jpg	44.jpg	66.jpg	122.jpg	146.jpg	97.jpg	
subject_9
57.jpg	82.jpg	9.jpg	106.jpg	48.jpg	39.jpg	63.jpg	79.jpg	29.jpg	31.jpg	27.jpg	75.jpg	103.jpg	32.jpg	41.jpg	47.jpg	42.jpg	45.jpg	87.jpg	53.jpg	86.jpg	10.jpg	19.jpg	111.jpg	35.jpg	7.jpg	26.jpg	67.jpg	52.jpg	49.jpg	101.jpg	44.jpg	66.jpg	97.jpg	
subject_17
48.jpg	39.jpg	160.jpg	207.jpg	63.jpg	131.jpg	79.jpg	29.jpg	31.jpg	27.jpg	114.jpg	75.jpg	148.jpg	103.jpg	32.jpg	41.jpg	167.jpg	175.jpg	119.jpg	176.jpg	208.jpg	188.jpg	162.jpg	159.jpg	158.jpg	47.jpg	121.jpg	42.jpg	45.jpg	198.jpg	172.jpg	87.jpg	184.jpg	53.jpg	201.jpg	86.jpg	178.jpg	235.jpg	10.jpg	1

In [ ]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [ ]:
# Save test and train data for later use

np.save('train_data_yolo_16_faces',x_train)
np.save('train_labels_yolo_16_faces',y_train)
np.save('test_data_yolo_16_faces',x_test)
np.save('test_labels_yolo_16_faces',y_test)



x_train=np.load('train_data_yolo_16_faces.npy')
y_train=np.load('train_labels_yolo_16_faces.npy')
x_test=np.load('test_data_yolo_16_faces.npy')
y_test=np.load('test_labels_yolo_16_faces.npy')

In [ ]:
# Softmax regressor to classify images based on encoding
print(x_train.shape)
classifier_model=Sequential()
classifier_model.add(Dense(units=100,input_dim=x_train.shape[1],kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.3))
classifier_model.add(Dense(units=6,kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(units=16,kernel_initializer='he_uniform'))
classifier_model.add(Activation('softmax'))
classifier_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='nadam',metrics=['accuracy'])

(1442, 2622)


In [ ]:
# Fit the model with the training dataset
classifier_model.fit(x_train,y_train,epochs=150,validation_data=(x_test,y_test))

Epoch 1/72
46/46 [==============================] - 2s 7ms/step - loss: 1.4187 - accuracy: 0.6990 - val_loss: 1.2124 - val_accuracy: 0.8339
Epoch 2/72
46/46 [==============================] - 0s 5ms/step - loss: 1.1378 - accuracy: 0.8190 - val_loss: 0.8095 - val_accuracy: 0.9617
Epoch 3/72
46/46 [==============================] - 0s 5ms/step - loss: 1.0252 - accuracy: 0.8675 - val_loss: 0.7562 - val_accuracy: 0.9952
Epoch 4/72
46/46 [==============================] - 0s 5ms/step - loss: 0.9147 - accuracy: 0.8953 - val_loss: 0.6761 - val_accuracy: 0.9984
Epoch 5/72
46/46 [==============================] - 0s 5ms/step - loss: 0.8110 - accuracy: 0.9119 - val_loss: 0.5617 - val_accuracy: 0.9984
Epoch 6/72
46/46 [==============================] - 0s 5ms/step - loss: 0.7650 - accuracy: 0.9147 - val_loss: 0.5379 - val_accuracy: 1.0000
Epoch 7/72
46/46 [==============================] - 0s 5ms/step - loss: 0.6957 - accuracy: 0.9105 - val_loss: 0.4726 - val_accuracy: 1.0000
Epoch 8/72
46/46 [==

46/46 [==============================] - 0s 5ms/step - loss: 0.1844 - accuracy: 0.9570 - val_loss: 0.0412 - val_accuracy: 1.0000
Epoch 60/72
46/46 [==============================] - 0s 5ms/step - loss: 0.1794 - accuracy: 0.9570 - val_loss: 0.0660 - val_accuracy: 0.9936
Epoch 61/72
46/46 [==============================] - 0s 6ms/step - loss: 0.1812 - accuracy: 0.9487 - val_loss: 0.0428 - val_accuracy: 1.0000
Epoch 62/72
46/46 [==============================] - 0s 6ms/step - loss: 0.1827 - accuracy: 0.9563 - val_loss: 0.0362 - val_accuracy: 1.0000
Epoch 63/72
46/46 [==============================] - 0s 5ms/step - loss: 0.1767 - accuracy: 0.9570 - val_loss: 0.0354 - val_accuracy: 1.0000
Epoch 64/72
46/46 [==============================] - 0s 5ms/step - loss: 0.1591 - accuracy: 0.9619 - val_loss: 0.0342 - val_accuracy: 1.0000
Epoch 65/72
46/46 [==============================] - 0s 6ms/step - loss: 0.1732 - accuracy: 0.9577 - val_loss: 0.0353 - val_accuracy: 1.0000
Epoch 66/72
46/46 [======

In [ ]:
tf.keras.models.save_model(classifier_model,path+'/face_classifier_model_yolo_16_faces.h5')

In [ ]:
classifier_model=tf.keras.models.load_model(path+'/face_classifier_model_yolo_16_faces.h5')#16 people

In [32]:
def plot(img):
  plt.figure(figsize=(8,4))
  plt.imshow(img[:,:,::-1])
  plt.show()

In [ ]:
from yoloface import face_analysis
face = face_analysis()

yolov3-tiny_face.weights:: status : file already exists
yolov3_tiny_face.cfg:: status : file already exists
face_detection.weights:: status : file already exists
face_detection.cfg:: status : file already exists


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# Định nghĩa model nhận diện khuôn mặt và phân loại danh tính
def detect_faces(image_path, output_path, face, vgg_face, classifier_model, person_rep):
    # Đọc ảnh
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Không tìm thấy ảnh: {image_path}")
        return

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Phát hiện khuôn mặt
    __, box, conf = face.face_detection(frame_arr=frame, frame_status=True, model='full')
    faces_rect = np.array([np.array(xi) for xi in box])

    for (x, y, w, h) in faces_rect:
        cv2.rectangle(frame, (x, y), (x+h, y+w), (0, 255, 0), thickness=2)

        # Cắt vùng khuôn mặt
        # img_crop = gray[y:y+w, x:x+h]
        img_crop = frame[y:y+w, x:x+h]

        try:
            cv2.imwrite(output_path + "/face.jpg", img_crop)
        except:
            continue

        # Tiền xử lý ảnh
        crop_img = load_img(output_path + "/face.jpg", target_size=(224, 224))
        crop_img = img_to_array(crop_img)
        crop_img = np.expand_dims(crop_img, axis=0)
        crop_img = preprocess_input(crop_img)

        # Trích xuất đặc trưng
        img_encode = vgg_face(crop_img)
        embed = K.eval(img_encode)

        # Dự đoán danh tính
        person = classifier_model.predict(embed)
        name = person_rep[np.argmax(person)]
        print(name)

        # Vẽ kết quả lên ảnh
        cv2.rectangle(gray, (x, y), (x+h, y+w), (0, 255, 0), 2)
        if np.max(person) > 0.80:
            cv2.putText(gray, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(gray, str(np.max(person)), (x, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

    # Hiển thị ảnh đã xử lý
    cv2_imshow(gray)
    cv2.imwrite(output_path + "/detected_faces.jpg", gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Sử dụng hàm
detect_faces("/content/images_test_crop/3/AUG_12.jpg", "/content/output", face, vgg_face, classifier_model, person_rep)
